In [4]:
!pip install opencv-python pyqt5

Defaulting to user installation because normal site-packages is not writeable


In [5]:
import sys
import os
import cv2
import numpy as np
from PyQt5.QtWidgets import (
    QApplication, QMainWindow, QWidget, QVBoxLayout, QHBoxLayout, QPushButton,
    QFileDialog, QLabel, QListWidget, QSlider, QMessageBox, QGroupBox, QGridLayout,
    QSpinBox, QComboBox, QScrollArea, QSplitter, QProgressBar
)
from PyQt5.QtGui import QPixmap, QImage, QPainter, QPen, QColor
from PyQt5.QtCore import Qt, QRect, pyqtSignal, QTimer, QPalette


class ImageItem:
    def __init__(self, path, img=None):
        self.path = path
        self.filename = os.path.basename(path)
        # Load the image if not provided
        self.original_img = cv2.imread(path) if img is None else img
        self.display_img = self.original_img.copy()
        self.crop_rect = None  # (x, y, w, h)
        self.aligned = False
        self.rotation_angle = 0

    def reset(self):
        self.display_img = self.original_img.copy()
        self.crop_rect = None
        self.aligned = False
        self.rotation_angle = 0

    def get_cropped_image(self):
        if self.crop_rect is None:
            return self.display_img
        x, y, w, h = self.crop_rect
        return self.display_img[y:y+h, x:x+w]


class ImagePreviewWidget(QWidget):
    cropChanged = pyqtSignal(QRect)

    def __init__(self, parent=None):
        super().__init__(parent)
        self.image = None
        self.display_pixmap = None
        self.scale_factor = 1.0
        self.crop_rect = None
        self.drag_start = None
        self.is_dragging = False
        self.enable_crop_selection = True

        self.setMinimumSize(500, 400)
        self.setMouseTracking(True)

    def set_image(self, img):
        self.image = img
        if img is not None:
            height, width = img.shape[:2]
            bytes_per_line = 3 * width
            q_img = QImage(img.data, width, height, bytes_per_line, QImage.Format_RGB888).rgbSwapped()
            self.display_pixmap = QPixmap.fromImage(q_img)
            self.scale_factor = min(self.width() / width, self.height() / height)
            self.crop_rect = None
        else:
            self.display_pixmap = None
        self.update()

    def resizeEvent(self, event):
        if self.image is not None:
            height, width = self.image.shape[:2]
            self.scale_factor = min(self.width() / width, self.height() / height)
        super().resizeEvent(event)

    def paintEvent(self, event):
        painter = QPainter(self)
        painter.fillRect(self.rect(), Qt.darkGray)

        if self.display_pixmap:
            img_width = int(self.display_pixmap.width() * self.scale_factor)
            img_height = int(self.display_pixmap.height() * self.scale_factor)
            x_offset = (self.width() - img_width) // 2
            y_offset = (self.height() - img_height) // 2
            target_rect = QRect(x_offset, y_offset, img_width, img_height)
            painter.drawPixmap(target_rect, self.display_pixmap)

            if self.crop_rect:
                painter.setPen(QPen(Qt.red, 2, Qt.SolidLine))
                x = int(self.crop_rect.x() * self.scale_factor) + x_offset
                y = int(self.crop_rect.y() * self.scale_factor) + y_offset
                w = int(self.crop_rect.width() * self.scale_factor)
                h = int(self.crop_rect.height() * self.scale_factor)
                painter.drawRect(x, y, w, h)

    def mousePressEvent(self, event):
        if not self.display_pixmap or not self.enable_crop_selection:
            return
        img_width = int(self.display_pixmap.width() * self.scale_factor)
        img_height = int(self.display_pixmap.height() * self.scale_factor)
        x_offset = (self.width() - img_width) // 2
        y_offset = (self.height() - img_height) // 2
        img_rect = QRect(x_offset, y_offset, img_width, img_height)
        if img_rect.contains(event.pos()):
            self.is_dragging = True
            self.drag_start = event.pos()
            x = int((event.x() - x_offset) / self.scale_factor)
            y = int((event.y() - y_offset) / self.scale_factor)
            self.crop_rect = QRect(x, y, 0, 0)

    def mouseMoveEvent(self, event):
        if self.is_dragging and self.crop_rect:
            img_width = int(self.display_pixmap.width() * self.scale_factor)
            img_height = int(self.display_pixmap.height() * self.scale_factor)
            x_offset = (self.width() - img_width) // 2
            y_offset = (self.height() - img_height) // 2

            current_x = max(0, min(int((event.x() - x_offset) / self.scale_factor), self.display_pixmap.width()))
            current_y = max(0, min(int((event.y() - y_offset) / self.scale_factor), self.display_pixmap.height()))
            start_x = self.crop_rect.x()
            start_y = self.crop_rect.y()

            width = current_x - start_x
            height = current_y - start_y

            if width < 0:
                start_x = current_x
                width = abs(width)
            if height < 0:
                start_y = current_y
                height = abs(height)

            self.crop_rect = QRect(start_x, start_y, width, height)
            self.update()

    def mouseReleaseEvent(self, event):
        if self.is_dragging:
            self.is_dragging = False
            if self.crop_rect and self.crop_rect.width() > 10 and self.crop_rect.height() > 10:
                self.cropChanged.emit(self.crop_rect)
            else:
                self.crop_rect = None
                self.update()


class ImageProcessingApp(QMainWindow):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("Image Crop and Align Tool")
        self.resize(1200, 800)

        self.images = []         # List of ImageItem
        self.current_idx = -1    # Currently selected image index
        self.template_img = None # Template image for alignment
        self.auto_align_method = "ORB"  # Default alignment method

        self.init_ui()

    def init_ui(self):
        self.apply_dark_theme()
        central_widget = QWidget()
        self.setCentralWidget(central_widget)
        main_layout = QVBoxLayout(central_widget)

        # Top buttons
        top_buttons = QHBoxLayout()
        self.btn_load = QPushButton("Load Images")
        self.btn_load.setMinimumHeight(36)
        self.btn_load.clicked.connect(self.load_images)
        top_buttons.addWidget(self.btn_load)

        self.btn_set_template = QPushButton("Set as Template")
        self.btn_set_template.setMinimumHeight(36)
        self.btn_set_template.clicked.connect(self.set_template)
        self.btn_set_template.setEnabled(False)
        top_buttons.addWidget(self.btn_set_template)

        self.btn_apply_to_all = QPushButton("Apply Crop to All")
        self.btn_apply_to_all.setMinimumHeight(36)
        self.btn_apply_to_all.clicked.connect(self.apply_crop_to_all)
        self.btn_apply_to_all.setEnabled(False)
        top_buttons.addWidget(self.btn_apply_to_all)

        self.btn_auto_align = QPushButton("Auto Align All")
        self.btn_auto_align.setMinimumHeight(36)
        self.btn_auto_align.clicked.connect(self.auto_align_all)
        self.btn_auto_align.setEnabled(False)
        top_buttons.addWidget(self.btn_auto_align)

        self.btn_save_all = QPushButton("Save All")
        self.btn_save_all.setMinimumHeight(36)
        self.btn_save_all.clicked.connect(self.save_all_images)
        self.btn_save_all.setEnabled(False)
        top_buttons.addWidget(self.btn_save_all)

        main_layout.addLayout(top_buttons)

        # Quick crop presets
        quick_crop_layout = QHBoxLayout()
        quick_crop_layout.addWidget(QLabel("Quick Crop:"))
        crop_presets = [
            ("1:1", (1, 1)),
            ("4:3", (4, 3)),
            ("16:9", (16, 9)),
            ("3:2", (3, 2)),
            ("5:4", (5, 4)),
            ("Custom", None)
        ]
        for label, ratio in crop_presets:
            btn = QPushButton(label)
            btn.setMinimumHeight(30)
            btn.clicked.connect(lambda checked, r=ratio: self.apply_crop_preset(r))
            quick_crop_layout.addWidget(btn)
        main_layout.addLayout(quick_crop_layout)

        # Splitter for main content
        splitter = QSplitter(Qt.Horizontal)

        # Left panel: image list and controls
        left_panel = QWidget()
        left_layout = QVBoxLayout(left_panel)

        list_group = QGroupBox("Images")
        list_layout = QVBoxLayout(list_group)
        self.image_list = QListWidget()
        self.image_list.setAlternatingRowColors(True)
        self.image_list.currentRowChanged.connect(self.image_selected)
        list_layout.addWidget(self.image_list)
        left_layout.addWidget(list_group)

        controls_group = QGroupBox("Controls")
        controls_layout = QGridLayout(controls_group)

        # Rotation controls
        controls_layout.addWidget(QLabel("Rotation:"), 0, 0)
        self.rotation_slider = QSlider(Qt.Horizontal)
        self.rotation_slider.setRange(-180, 180)
        self.rotation_slider.setValue(0)
        self.rotation_slider.setTickPosition(QSlider.TicksBelow)
        self.rotation_slider.setTickInterval(30)
        self.rotation_slider.valueChanged.connect(self.rotate_image)
        controls_layout.addWidget(self.rotation_slider, 0, 1)
        self.rotation_value = QLabel("0°")
        controls_layout.addWidget(self.rotation_value, 0, 2)

        # Fine rotation buttons
        fine_rotation_layout = QHBoxLayout()
        self.btn_rotate_left = QPushButton("← 0.5°")
        self.btn_rotate_left.clicked.connect(lambda: self.fine_rotate(-0.5))
        fine_rotation_layout.addWidget(self.btn_rotate_left)
        self.btn_rotate_right = QPushButton("0.5° →")
        self.btn_rotate_right.clicked.connect(lambda: self.fine_rotate(0.5))
        fine_rotation_layout.addWidget(self.btn_rotate_right)
        controls_layout.addLayout(fine_rotation_layout, 1, 0, 1, 3)

        # Reset current image button
        self.btn_reset = QPushButton("Reset Current Image")
        self.btn_reset.clicked.connect(self.reset_current_image)
        self.btn_reset.setEnabled(False)
        controls_layout.addWidget(self.btn_reset, 2, 0, 1, 3)

        # Alignment method selection
        controls_layout.addWidget(QLabel("Alignment Method:"), 3, 0)
        self.align_method = QComboBox()
        self.align_method.addItems(["ORB", "SIFT", "ECC"])
        self.align_method.currentTextChanged.connect(self.set_align_method)
        controls_layout.addWidget(self.align_method, 3, 1, 1, 2)

        # Crop tip
        crop_help = QLabel("Crop tip: Click and drag to define crop area")
        crop_help.setStyleSheet("color: #7F9DB9; font-style: italic;")
        controls_layout.addWidget(crop_help, 4, 0, 1, 3)

        # Fine translation controls
        translation_group = QGroupBox("Fine Translation")
        translation_layout = QGridLayout(translation_group)
        translation_layout.addWidget(QLabel("X:"), 0, 0)
        self.x_translation = QSpinBox()
        self.x_translation.setRange(-100, 100)
        self.x_translation.setValue(0)
        self.x_translation.setSingleStep(1)
        self.x_translation.valueChanged.connect(self.apply_translation)
        translation_layout.addWidget(self.x_translation, 0, 1)
        translation_layout.addWidget(QLabel("Y:"), 1, 0)
        self.y_translation = QSpinBox()
        self.y_translation.setRange(-100, 100)
        self.y_translation.setValue(0)
        self.y_translation.setSingleStep(1)
        self.y_translation.valueChanged.connect(self.apply_translation)
        translation_layout.addWidget(self.y_translation, 1, 1)
        controls_layout.addWidget(translation_group, 5, 0, 1, 3)

        left_layout.addWidget(controls_group)

        # Progress bar for long operations
        self.progress_bar = QProgressBar()
        self.progress_bar.setStyleSheet("QProgressBar { text-align: center; }")
        self.progress_bar.setVisible(False)
        left_layout.addWidget(self.progress_bar)

        splitter.addWidget(left_panel)

        # Right panel: image preview
        right_panel = QWidget()
        right_layout = QVBoxLayout(right_panel)
        self.image_preview = ImagePreviewWidget()
        self.image_preview.cropChanged.connect(self.crop_changed)
        scroll_area = QScrollArea()
        scroll_area.setWidget(self.image_preview)
        scroll_area.setWidgetResizable(True)
        right_layout.addWidget(scroll_area)
        splitter.addWidget(right_panel)
        splitter.setSizes([int(self.width() * 0.3), int(self.width() * 0.7)])
        main_layout.addWidget(splitter)

        self.statusBar().setStyleSheet("QStatusBar { border-top: 1px solid #444; }")
        self.statusBar().showMessage("Ready")

    def apply_dark_theme(self):
        dark_palette = self.palette()
        dark_color = QColor(45, 45, 45)
        disabled_color = QColor(70, 70, 70)
        text_color = QColor(210, 210, 210)
        highlight_color = QColor(42, 130, 218)
        dark_palette.setColor(QPalette.Window, dark_color)
        dark_palette.setColor(QPalette.WindowText, text_color)
        dark_palette.setColor(QPalette.Base, QColor(25, 25, 25))
        dark_palette.setColor(QPalette.AlternateBase, dark_color)
        dark_palette.setColor(QPalette.ToolTipBase, text_color)
        dark_palette.setColor(QPalette.ToolTipText, text_color)
        dark_palette.setColor(QPalette.Text, text_color)
        dark_palette.setColor(QPalette.Disabled, QPalette.Text, QColor(150, 150, 150))
        dark_palette.setColor(QPalette.Button, dark_color)
        dark_palette.setColor(QPalette.ButtonText, text_color)
        dark_palette.setColor(QPalette.Disabled, QPalette.ButtonText, QColor(150, 150, 150))
        dark_palette.setColor(QPalette.BrightText, Qt.red)
        dark_palette.setColor(QPalette.Link, highlight_color)
        dark_palette.setColor(QPalette.Highlight, highlight_color)
        dark_palette.setColor(QPalette.HighlightedText, Qt.black)
        dark_palette.setColor(QPalette.Disabled, QPalette.Highlight, disabled_color)
        self.setPalette(dark_palette)
        self.setStyleSheet("""
            QMainWindow, QWidget {
                background-color: #2D2D2D;
                color: #D2D2D2;
            }
            QGroupBox {
                border: 1px solid #555;
                border-radius: 5px;
                margin-top: 10px;
                font-weight: bold;
                padding-top: 10px;
            }
            QGroupBox::title {
                subcontrol-origin: margin;
                left: 10px;
                padding: 0 3px;
            }
            QPushButton {
                background-color: #444;
                border: none;
                border-radius: 4px;
                padding: 5px 10px;
            }
            QPushButton:hover {
                background-color: #555;
            }
            QPushButton:pressed {
                background-color: #666;
            }
            QPushButton:disabled {
                background-color: #333;
                color: #777;
            }
            QComboBox {
                background-color: #444;
                border: 1px solid #555;
                border-radius: 3px;
                padding: 2px 8px;
            }
            QComboBox:hover {
                background-color: #555;
            }
            QListWidget {
                background-color: #222;
                alternate-background-color: #2A2A2A;
                border: 1px solid #444;
                border-radius: 3px;
            }
            QSlider::groove:horizontal {
                border: 1px solid #999;
                height: 6px;
                background: #444;
                margin: 2px 0;
                border-radius: 3px;
            }
            QSlider::handle:horizontal {
                background: #5A8CBF;
                border: 1px solid #5A8CBF;
                width: 14px;
                margin: -4px 0;
                border-radius: 7px;
            }
            QProgressBar {
                border: 1px solid #555;
                border-radius: 3px;
                text-align: center;
                background-color: #333;
            }
            QProgressBar::chunk {
                background-color: #5A8CBF;
                width: 1px;
            }
            QLineEdit, QSpinBox {
                background-color: #333;
                border: 1px solid #555;
                border-radius: 3px;
                padding: 2px 4px;
            }
            QScrollBar:vertical {
                border: none;
                background: #333;
                width: 12px;
                margin: 12px 0 12px 0;
                border-radius: 0px;
            }
            QScrollBar::handle:vertical {
                background: #555;
                min-height: 20px;
                border-radius: 6px;
            }
            QScrollBar::add-line:vertical, QScrollBar::sub-line:vertical {
                background: none;
            }
        """)

    def load_images(self):
        files, _ = QFileDialog.getOpenFileNames(
            self, "Select Images", "", "Image Files (*.png *.jpg *.jpeg *.bmp *.tif *.tiff)"
        )
        if not files:
            return

        self.images = []
        self.image_list.clear()
        self.current_idx = -1
        self.template_img = None

        self.progress_bar.setVisible(True)
        self.progress_bar.setRange(0, len(files))
        self.progress_bar.setValue(0)

        for i, file_path in enumerate(files):
            try:
                self.images.append(ImageItem(file_path))
                self.image_list.addItem(os.path.basename(file_path))
                self.progress_bar.setValue(i + 1)
                QApplication.processEvents()  # Refresh UI
            except Exception as e:
                QMessageBox.warning(self, "Error", f"Failed to load {file_path}: {str(e)}")
        self.progress_bar.setVisible(False)

        if self.images:
            self.image_list.setCurrentRow(0)
            self.btn_set_template.setEnabled(True)
            self.btn_save_all.setEnabled(True)
            self.btn_reset.setEnabled(True)
            self.statusBar().showMessage(f"Loaded {len(self.images)} images")

    def apply_crop_preset(self, ratio):
        if self.current_idx < 0 or ratio is None:
            return

        img_item = self.images[self.current_idx]
        img = img_item.display_img
        height, width = img.shape[:2]

        # Calculate crop rectangle based on ratio (centered crop)
        if ratio[0] > ratio[1]:  # Landscape
            target_h = int(width * ratio[1] / ratio[0])
            if target_h > height:
                target_w = int(height * ratio[0] / ratio[1])
                x = (width - target_w) // 2
                y = 0
                w = target_w
                h = height
            else:
                x = 0
                y = (height - target_h) // 2
                w = width
                h = target_h
        else:  # Portrait or square
            target_w = int(height * ratio[0] / ratio[1])
            if target_w > width:
                target_h = int(width * ratio[1] / ratio[0])
                x = 0
                y = (height - target_h) // 2
                w = width
                h = target_h
            else:
                x = (width - target_w) // 2
                y = 0
                w = target_w
                h = height

        # Apply crop rectangle
        img_item.crop_rect = (x, y, w, h)
        self.image_preview.crop_rect = QRect(x, y, w, h)
        self.image_preview.update()
        self.btn_apply_to_all.setEnabled(True)

    def fine_rotate(self, angle_delta):
        if self.current_idx < 0:
            return
        current_angle = self.images[self.current_idx].rotation_angle
        new_angle = current_angle + angle_delta
        self.rotation_slider.setValue(int(new_angle))
        # Call rotate_image to update using the new angle
        self.rotate_image(new_angle)

    def rotate_image(self, value):
        if self.current_idx < 0:
            return
        img_item = self.images[self.current_idx]
        img_item.rotation_angle = value
        self.rotation_value.setText(f"{value:.1f}°")
        height, width = img_item.original_img.shape[:2]
        center = (width // 2, height // 2)
        rotation_matrix = cv2.getRotationMatrix2D(center, value, 1.0)
        cos = np.abs(rotation_matrix[0, 0])
        sin = np.abs(rotation_matrix[0, 1])
        new_width = int((height * sin) + (width * cos))
        new_height = int((height * cos) + (width * sin))
        rotation_matrix[0, 2] += (new_width / 2) - center[0]
        rotation_matrix[1, 2] += (new_height / 2) - center[1]
        rotated = cv2.warpAffine(img_item.original_img, rotation_matrix, (new_width, new_height))
        img_item.display_img = rotated
        # Clear crop rectangle after rotation
        img_item.crop_rect = None
        self.image_preview.crop_rect = None
        self.image_preview.set_image(rotated)

    def apply_translation(self):
        if self.current_idx < 0:
            return
        x_trans = self.x_translation.value()
        y_trans = self.y_translation.value()
        img_item = self.images[self.current_idx]
        M = np.float32([[1, 0, x_trans], [0, 1, y_trans]])
        h, w = img_item.display_img.shape[:2]
        translated = cv2.warpAffine(img_item.display_img, M, (w, h))
        img_item.display_img = translated
        self.image_preview.set_image(translated)

    def reset_current_image(self):
        if self.current_idx < 0:
            return
        self.images[self.current_idx].reset()
        self.rotation_slider.setValue(0)
        self.rotation_value.setText("0°")
        self.image_preview.set_image(self.images[self.current_idx].display_img)

    def set_align_method(self, method):
        self.auto_align_method = method

    def set_template(self):
        if self.current_idx < 0:
            return
        self.template_img = self.images[self.current_idx]
        self.statusBar().showMessage(f"Template set: {self.template_img.filename}")
        self.btn_auto_align.setEnabled(True)

    def image_selected(self, index):
        self.current_idx = index
        if index < 0 or index >= len(self.images):
            return
        img_item = self.images[index]
        self.image_preview.set_image(img_item.display_img)
        self.rotation_slider.setValue(int(img_item.rotation_angle))
        self.rotation_value.setText(f"{img_item.rotation_angle:.1f}°")

    def crop_changed(self, rect):
        if self.current_idx < 0:
            return
        # Update the crop rectangle as a tuple (x, y, width, height)
        self.images[self.current_idx].crop_rect = (rect.x(), rect.y(), rect.width(), rect.height())

    def apply_crop_to_all(self):
        if self.current_idx < 0:
            return
        current_crop = self.images[self.current_idx].crop_rect
        if current_crop is None:
            return
        # Option: apply crop only to images sharing a name prefix
        # For now, apply crop to all images in the list
        x, y, w, h = current_crop
        for img_item in self.images:
            try:
                img_item.crop_rect = current_crop
                # Note: This assumes images have similar dimensions (as with PBR sets)
                img_item.display_img = img_item.display_img[y:y+h, x:x+w]
            except Exception as e:
                print(f"Error applying crop to {img_item.filename}: {e}")
        # Refresh preview for current image
        self.image_preview.set_image(self.images[self.current_idx].display_img)

    def auto_align_all(self):
        if not self.template_img:
            QMessageBox.warning(self, "Error", "Please set a template image first")
            return

        template = self.template_img.display_img
        self.progress_bar.setVisible(True)
        self.progress_bar.setRange(0, len(self.images))
        self.progress_bar.setValue(0)
        method = self.auto_align_method

        for i, img_item in enumerate(self.images):
            if img_item != self.template_img:
                try:
                    template_gray = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)
                    img_gray = cv2.cvtColor(img_item.display_img, cv2.COLOR_BGR2GRAY)
                    if method == "ORB":
                        orb = cv2.ORB_create()
                        kp1, des1 = orb.detectAndCompute(template_gray, None)
                        kp2, des2 = orb.detectAndCompute(img_gray, None)
                        bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
                        matches = bf.match(des1, des2)
                        matches = sorted(matches, key=lambda x: x.distance)
                        good_matches = matches[:min(50, len(matches))]
                        if len(good_matches) >= 10:
                            src_pts = np.float32([kp1[m.queryIdx].pt for m in good_matches]).reshape(-1, 1, 2)
                            dst_pts = np.float32([kp2[m.trainIdx].pt for m in good_matches]).reshape(-1, 1, 2)
                            transform_matrix, _ = cv2.estimateAffinePartial2D(dst_pts, src_pts, method=cv2.RANSAC)
                            if transform_matrix is not None:
                                h, w = template.shape[:2]
                                aligned = cv2.warpAffine(img_item.display_img, transform_matrix, (w, h))
                                img_item.display_img = aligned
                                img_item.aligned = True
                    elif method == "SIFT":
                        try:
                            sift = cv2.SIFT_create()
                            kp1, des1 = sift.detectAndCompute(template_gray, None)
                            kp2, des2 = sift.detectAndCompute(img_gray, None)
                            FLANN_INDEX_KDTREE = 1
                            index_params = dict(algorithm=FLANN_INDEX_KDTREE, trees=5)
                            search_params = dict(checks=50)
                            flann = cv2.FlannBasedMatcher(index_params, search_params)
                            matches = flann.knnMatch(des1, des2, k=2)
                            good_matches = []
                            for m, n in matches:
                                if m.distance < 0.7 * n.distance:
                                    good_matches.append(m)
                            if len(good_matches) >= 10:
                                src_pts = np.float32([kp1[m.queryIdx].pt for m in good_matches]).reshape(-1, 1, 2)
                                dst_pts = np.float32([kp2[m.trainIdx].pt for m in good_matches]).reshape(-1, 1, 2)
                                transform_matrix, _ = cv2.estimateAffinePartial2D(dst_pts, src_pts, method=cv2.RANSAC)
                                if transform_matrix is not None:
                                    h, w = template.shape[:2]
                                    aligned = cv2.warpAffine(img_item.display_img, transform_matrix, (w, h))
                                    img_item.display_img = aligned
                                    img_item.aligned = True
                        except Exception as e:
                            print(f"SIFT alignment error for {img_item.filename}: {e}")
                    elif method == "ECC":
                        try:
                            template_gray_float = template_gray.astype(np.float32) / 255.0
                            img_gray_float = img_gray.astype(np.float32) / 255.0
                            warp_matrix = np.eye(2, 3, dtype=np.float32)
                            criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 50, 1e-5)
                            cc, warp_matrix = cv2.findTransformECC(template_gray_float, img_gray_float, warp_matrix, cv2.MOTION_EUCLIDEAN, criteria)
                            if warp_matrix is not None:
                                h, w = template.shape[:2]
                                aligned = cv2.warpAffine(img_item.display_img, warp_matrix, (w, h), flags=cv2.INTER_LINEAR + cv2.WARP_INVERSE_MAP)
                                img_item.display_img = aligned
                                img_item.aligned = True
                        except Exception as e:
                            print(f"ECC alignment error for {img_item.filename}: {e}")
                except Exception as e:
                    print(f"Error aligning {img_item.filename}: {e}")
            self.progress_bar.setValue(i + 1)
            QApplication.processEvents()
        self.progress_bar.setVisible(False)
        self.statusBar().showMessage("Auto alignment complete")

    def save_all_images(self):
        directory = QFileDialog.getExistingDirectory(self, "Select Directory to Save Images")
        if not directory:
            return
        for img_item in self.images:
            base, ext = os.path.splitext(img_item.filename)
            save_path = os.path.join(directory, base + "_processed" + ext)
            cv2.imwrite(save_path, img_item.display_img)
        QMessageBox.information(self, "Saved", "All images have been saved.")


if __name__ == "__main__":
    app = QApplication(sys.argv)
    window = ImageProcessingApp()
    window.show()
    sys.exit(app.exec_())


ImportError: cannot import name 'QPalette' from 'PyQt5.QtCore' (c:\ProgramData\anaconda3\Lib\site-packages\PyQt5\QtCore.pyd)